In [1]:
import numpy as np  # probably don't need to load
import pandas as pd
import datetime as dt
#import pandas_datareader.data as web  # probably don't need to load
#import quandl
import blpapi
from xbbg import blp

import matplotlib.pyplot as plt
#import seaborn as sns

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Start with reading Jeff's Master sheet
- this comes from BMS (onex.blkmtn.com/?db=238)  columns B:BD
- columns BA to BD of that tab to calculate the WARF using the new logic that Moody’s released (see new WARF section)
- The CLO 21 Port as of 3.18 tab contains the actual portfolio of CLO 21 as of 3/18 which also feeds into the Model Portfolio tab (column H and K).

In [3]:
filepath = 'Z:/Shared/Risk Management and Investment Technology/Files for sharing/'
file = 'CLO21 model portfolio as of 03.18.21 - With New WARF Logic.xlsm'

# derived data in MASTER are as follows:
# column A 'combined' is just concat('Issuer','-','Asset')
# all the above is trivial to add later when spreadsheet read gets replaced
# by reading directly from BMS

master_df = pd.read_excel(filepath + file,sheet_name='MASTER',header=1)
master_df = master_df.loc[:,~master_df.columns.str.match("Unnamed")]

In [4]:
# looks like LoanX ID makes the most sense as the index
master_df

,Combined,LoanX ID,Parent Company,Issuer,Asset,Analyst,Floating Spread,Floating Spread Floor,All In Rate,Maturity Date,...,Issuer Country,Cov Lite,Default,Libor Contract,Loan Only,Comments,Moody's ADJ. CFR Score,Adjusted CFR Score,Adjusted CFR for WARF,Adj. WARF NEW
0,CNT Holdings I Corp - Initial Term Loan (Secon...,LX190245,1-800 Contacts,CNT Holdings I Corp,Initial Term Loan (Second Lien),Adey Delbridge,0.0675,0.0075,0.075000,2028-11-06,...,US,No,N,NaN,NaN,NaN,16,16,B3,3490
1,CNT Holdings I Corp - Initial Term Loan (First...,LX190219,1-800 Contacts,CNT Holdings I Corp,Initial Term Loan (First Lien),Adey Delbridge,0.0375,0.0075,0.045000,2027-11-08,...,US,No,N,NaN,NaN,NaN,16,16,B3,3490
2,1011778 B.C. Unlimited Liability Company (New ...,LX182732,1011778 B.C. Unlimited Liability Company (New ...,1011778 B.C. Unlimited Liability Company (New ...,Term A Loan,Rekha Nayar,0.0125,NaN,0.014913,2024-09-06,...,CA,No,N,NaN,NaN,NaN,13,13,Ba3,1766
3,1011778 B.C. Unlimited Liability Company (New ...,USC6900PAL34,1011778 B.C. Unlimited Liability Company (New ...,1011778 B.C. Unlimited Liability Company (New ...,3.500% - 02/2029 - USC6900PAL34 REGS,Rekha Nayar,NaN,NaN,0.035000,2029-02-15,...,CA,No,N,NaN,NaN,NaN,13,13,Ba3,1766
4,1011778 B.C. Unlimited Liability Company (New ...,LX183929,1011778 B.C. Unlimited Liability Company (New ...,1011778 B.C. Unlimited Liability Company (New ...,Term B-4 Loan,Rekha Nayar,0.0175,0.0000,0.019913,2026-11-19,...,CA,No,N,NaN,NaN,NaN,13,13,Ba3,1766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,zz_LXREP22 - Rep Line - 22,LXREP22,LXREP22,zz_LXREP22,Rep Line - 22,NaN,0.0425,0.0075,NaN,2029-03-08,...,US,No,N,NaN,NaN,NaN,16,16,B3,3490
492,zz_LXREP23 - Rep Line - 23,LXREP23,LXREP23,zz_LXREP23,Rep Line - 23,NaN,0.0450,0.0000,NaN,2029-03-09,...,US,No,N,NaN,NaN,NaN,16,16,B3,3490
493,zz_LXREP24 - Rep Line - 24,LXREP24,LXREP24,zz_LXREP24,Rep Line - 24,NaN,0.0450,0.0075,NaN,2029-03-10,...,US,No,N,NaN,NaN,NaN,16,16,B3,3490
494,zz_LXREP25 - Rep Line - 25,LXREP25,LXREP25,zz_LXREP25,Rep Line - 25,NaN,0.0475,0.0075,NaN,2029-03-11,...,US,No,N,NaN,NaN,NaN,16,16,B3,3490


# Read CLO 21 Port as of 3.18

In [5]:
CLO_df = pd.read_excel(filepath + file,sheet_name='CLO 21 Port as of 3.18',header=6,usecols='A:K')
CLO_df.dropna(inplace=True)

# his pivot table columns M:0
CLO_df[['Cusip or LIN','Quantity','/Unit']].sort_values(by='Cusip or LIN')

,Cusip or LIN,Quantity,/Unit
111,LX127674,886324.28,1.0035
30,LX152766,2500000.00,0.9900
92,LX155959,1771101.45,1.0008
55,LX159621,3000000.00,1.0012
6,LX162006,2000000.00,0.9950
...,...,...,...
0,LX193380,2000000.00,0.9900
120,LX193398,1500000.00,0.9950
24,LX193419,1500000.00,0.9900
51,LX193422,1000000.00,1.0000


In [6]:
CLO_df[['Quantity']].sum()  # verified sum of Quantity
CLO_df[['/Unit']].mean()    # verified for average /Unit

Quantity    2.263333e+08
dtype: float64

/Unit    0.997052
dtype: float64

# Read Bid.Ask 3.18

In [7]:
# need to use str.match so it isn't date dependent
bidask_df = pd.read_excel(filepath + file,sheet_name='Bid.Ask 3.18',header=0)
bidask_df = bidask_df.loc[:,~bidask_df.columns.str.match("Unnamed")]

# new WARF
- column BA looks up the 'CFR' score e.g. B3 and maps it to a number
- column BB adjusts +/- 1 if outlook is possible up/downgrade
- column BC converts back into an alpanumeric e.g. B3
- column BD maps the alphanumeric into the new WARF score (1-1000)

In [8]:
moodys_score = pd.read_excel(filepath + file,sheet_name='New WARF',header=0,usecols='E:F')
moodys_rfTable = pd.read_excel(filepath + file,sheet_name='New WARF',header=0,usecols='J:K')

In [87]:
# this works; spot checked
def moodys_adjusted_warf(df,moodys_score,moodys_rfTable):
    """
    This function creates the new Moody's Ratings Factor based 
    on the old Moody's rating.
    
    Arg in:
        df: the input data frame (from the MASTER table d/l'd from BMS)
        moodys_score: dataframe with alphanumeric rating to numeric map (1 to 1 map; linear)
        moodys_rfTable: dataframe with alphanumeric rating to new WARF numeric (1 to 1 map; 1 to 1000 values)
    """
    score = df['Moody\'s CFR'].map(dict(moodys_score[['Moodys','Score']].values))
    updown = df['Moody\'s Issuer Watch'].\
        apply(lambda x: -1 if x == 'Possible Upgrade' else 1 if x == 'Possible Downgrade' else 0)
    aScore = score + updown
    Adjusted_CFR_for_WARF = aScore.map(dict(moodys_score[['Score','Moodys']].values))
    # I keep the same column name as Jeff to make it easier to double check values
    df['Adj. WARF NEW'] = Adjusted_CFR_for_WARF.map(dict(moodys_rfTable[['Moody\'s Rating Factor Table','Unnamed: 10']].values))
    return df

In [88]:
master_df = moodys_adjusted_warf(master_df,moodys_score,moodys_rfTable)
master_df['Adj. WARF NEW'].sum()

1425500.0

In [113]:
model_port = master_df.merge(CLO_df,left_on="LoanX ID",right_on="Cusip or LIN",how='outer') 


In [115]:
#model_port = 
model_port.merge(bidask_df,left_on="LoanX ID",right_on="LXID",how='left').info()
#model_port

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 0 to 495
Data columns (total 79 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Combined                           496 non-null    object        
 1   LoanX ID                           496 non-null    object        
 2   Parent Company                     496 non-null    object        
 3   Issuer                             496 non-null    object        
 4   Asset                              496 non-null    object        
 5   Analyst                            462 non-null    object        
 6   Floating Spread                    488 non-null    float64       
 7   Floating Spread Floor              473 non-null    float64       
 8   All In Rate                        470 non-null    float64       
 9   Maturity Date                      494 non-null    datetime64[ns]
 10  Mark Price                         463

# S&P's Recovery Rate

In [12]:
#first_lien_rr = pd.read_excel(filepath + file, sheet_name='SP RR Updated', header=1, usecols='A:D')
#second_lien_rr = pd.read_excel(filepath + file, sheet_name='SP RR Updated', header=1, usecols='F:I')
new_sp_rr = pd.read_excel(filepath + file, sheet_name='SP RR Updated', header=1, usecols='L:M')
new_sp_rr.dropna(how='all',inplace=True)

lien_rr = pd.read_excel(filepath + file, sheet_name='SP RR Updated', header=1, usecols='A:I')
lien_rr.dropna(how='all',inplace=True)

bond_split = lien_rr[lien_rr['Country.1']=='Bonds'].index.values[0]
bond_table = lien_rr.loc[bond_split+1:]
lien_rr = lien_rr.loc[:bond_split-1]
lien_rr.drop(columns=['Unnamed: 4','Country Abv.1','Country.1','Group.1'],inplace=True)
lien_rr.rename(columns={'RR.1':'RR.2nd'},inplace=True)

In [13]:
def sp_recovery_rate(model_df,lien,new_rr,bond_table):
    """
    This function get the S&P recovery rate as a percent. If it doesn't exist
    in the master field, it will look up in the appropriate first and second 
    lien tables, if not, will look up the bond table.
    
    Arg in:
        model_df: the input data frame (from the MASTER table d/l'd from BMS)
        lien: a DF table with the RR's for first and second lien by country
        new_rr: a df mapping of the old notation for RR to a new RR in percentage
        bond_table: split out of a table for RR for bonds
    Arg out:
        model_df with inserted new column 'S&P Recovery Rate (AAA)'
    """
     
    # if it the Recovery rate exists lookup in AAA table
    model_df['S&P Recovery Rate (AAA)'] = model_df['S&P Recovery'].\
        map(dict(new_sp_rr[['S&P Recovery Rating\nand Recovery\nIndicator of\nCollateral Obligations','“AAA”']].values))
    
    # doesn't exist, but first lien, use first lien table
    model_df.loc[pd.isna(model_df['S&P Recovery']) & (model_df['Lien Type']== 'First Lien'),'S&P Recovery Rate (AAA)'] =\
        model_df.loc[pd.isna(model_df['S&P Recovery']) & (model_df['Lien Type']== 'First Lien'),'Issuer Country'].\
        map(dict(lien_rr[['Country Abv','RR']].values))
    
    
    # doesn't exist, but 2nd lien, use 2nd lien table
    model_df.loc[pd.isna(model_df['S&P Recovery']) & (model_df['Lien Type']== 'Second Lien'),'S&P Recovery Rate (AAA)'] = \
        model_df.loc[pd.isna(model_df['S&P Recovery']) & (model_df['Lien Type']== 'Second Lien'),'Issuer Country'].\
        map(dict(lien_rr[['Country Abv','RR.2nd']].values))
    
    # the bonds
    model_df.loc[pd.isna(model_df['S&P Recovery']) & pd.isna(model_df['Lien Type']),'S&P Recovery Rate (AAA)'] = \
        model_df.loc[pd.isna(model_df['S&P Recovery']) & pd.isna(model_df['Lien Type']),'Issuer Country'].\
        map(dict(bond_table[['Country Abv.1','RR.1']].values))

    return model_df

In [92]:
model_port = sp_recovery_rate(model_port,lien_rr,new_sp_rr,bond_table)
model_port.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496 entries, 0 to 495
Data columns (total 68 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Combined                           496 non-null    object        
 1   LoanX ID                           496 non-null    object        
 2   Parent Company                     496 non-null    object        
 3   Issuer                             496 non-null    object        
 4   Asset                              496 non-null    object        
 5   Analyst                            462 non-null    object        
 6   Floating Spread                    488 non-null    float64       
 7   Floating Spread Floor              473 non-null    float64       
 8   All In Rate                        470 non-null    float64       
 9   Maturity Date                      494 non-null    datetime64[ns]
 10  Mark Price                         463

# Diversity Score

In [39]:
model_port['Par_no_default'] = model_port['Quantity']
model_port.loc[model_port['Default']=='Y','Par_no_default'] = 0

div_df = model_port[['Parent Company','Moody\'s Industry','Par_no_default']].copy()
div_df.sort_values(by='Moody\'s Industry',inplace=True)


Par_no_default    1.782152e+06
dtype: float64

Par_no_default    2.263333e+08
dtype: float64

Parent Company    366
dtype: int64

In [18]:
ind_avg_eu = pd.read_excel(filepath + file, sheet_name='Diversity', header=8, usecols='K:L')
ind_avg_eu.dropna(how='all',inplace=True)
ind_avg_eu

,Aggregate\nIndustry\nEquivalent\nUnit Score,Industry\nDiversity\nScore
0,0.00,0.00
1,0.05,0.10
2,0.15,0.20
3,0.25,0.30
4,0.35,0.40
...,...,...
196,19.55,4.96
197,19.65,4.97
198,19.75,4.98
199,19.85,4.99


In [59]:
def diversity_score(model_df, ind_avg_eu):
    """
    This function calculates the Moody's Industry Diversity Score for the CLO
    
    Arg in:
        model_df: the input data frame (from the MASTER table d/l'd from BMS)
        ind_avg_eu: Moody's discrete lookup table that maps AIEUS to IDS, need to be sorted
    Arg out:
        dscore: the scalar measure of the IDS
    """
    
    #first create the Par amount filtering out defaults
    model_port['Par_no_default'] = model_port['Quantity']
    model_port.loc[model_port['Default']=='Y','Par_no_default'] = 0
    div_df = model_port[['Parent Company','Moody\'s Industry','Par_no_default']].copy()
    div_df.sort_values(by='Moody\'s Industry',inplace=True)

    # this keeps the industry, but groups on parent company for multiple loans
    test = div_df.groupby(by=['Parent Company','Moody\'s Industry']).sum()
    avg_par_amt = test.sum()/test.count()   
    
    # create the EU score for each parent
    # Lesser of 1 and Issuer Par Amount for such issuer divided by the Average Par Amount.
    test['EU'] = test[['Par_no_default']]/test[['Par_no_default']].mean()
    test.loc[test['EU']>1,'EU']=1
    
    # groupby Industry for the Ind Div Score
    IDS = test.groupby(by=['Moody\'s Industry']).sum()

    # this is like vlookup(..,TRUE) where the nearest match on merge is used, direction controls how
    # backward is the lesser if EU falls between AIEUS marks
    df_merged = pd.merge_asof(IDS.sort_values('EU'), ind_avg_eu, left_on='EU', 
                          right_on='Aggregate\nIndustry\nEquivalent\nUnit Score', direction='backward', suffixes=['', '_2'])
    dscore = df_merged['Industry\nDiversity\nScore'].sum()
    return dscore

In [93]:
diversity_score(model_port, ind_avg_eu)

62.726699999999994

# Model Portfolio stats
- Estimated Libor
- Minimum Floating Spread Test - Without Libor Floors
- Minimum Floating Spread Test - WithLibor Floors (adj. All in Rate)
- Maximum Moody's Rating Factor Test
- Maximum Moody's Rating Factor Test (NEW WARF)
- Maximum Moody's Rating Factor Test (Orig WARF)
- Minimum Weighted Average Moody's Recovery Rate Test
- Minimum Weighted Average S&P Recovery Rate Class A-1a
- Moody's Diversity Test
- WAP (Current Positions use Actual purchase price, all others use Ask price)
- Total Portfolio Par (excluding Defaults)
- Total Portfolio Par
- Current Portfolio 

- Replines	
- Amount	$79.2 
- WAS	3.80%
- WAPP	 99.5 
- WARF New	 3,022 
- WARF Orig	 2,954 


In [103]:
LIBOR = .0020
model_port['Adj. All in Rate'] = \
    model_port[['Floating Spread','Floating Spread Floor']].apply(lambda x: (x[0]+x[1]-.002) if (x[1]>.002) else x[0],axis=1 )
model_port['Adj. All in Rate']

0      0.0730
1      0.0430
2      0.0125
3         NaN
4      0.0175
        ...  
491    0.0480
492    0.0450
493    0.0505
494    0.0530
495    0.0780
Name: Adj. All in Rate, Length: 496, dtype: float64

In [110]:
model_port['Quantity'].sum()

226333336.85

In [111]:
def Port_stats(model_df):
    Port_stats_df = pd.DataFrame(np.nan,index=['Min Floating Spread Test - no Libor Floors',
        'Min Floating Spread Test - With Libor Floors',
        'Max Moodys Rating Factor Test (NEW WARF)',
        'Max Moodys Rating Factor Test (Orig WARF)',
        'Min Moodys Recovery Rate Test',
        'Min S&P Recovery Rate Class A-1a',
        'Moodys Diversity Test',
        'WAP',
        'Total Portfolio Par (excl. Defaults)',
        'Total Portfolio Par',
        'Current Portfolio'],columns = ['Portfolio Stats'])
    #model_df['Par_no_default'] = model_df['Quantity']
    #model_df.loc[model_df['Default']=='Y','Par_no_default'] = 0
    
    Port_stats_df.loc['Min Floating Spread Test - no Libor Floors','Portfolio Stats'] = \
        (model_df['Par_no_default']*model_df['Floating Spread']).sum()/model_df['Par_no_default'].sum()
    Port_stats_df.loc['Min Floating Spread Test - With Libor Floors','Portfolio Stats'] = \
        (model_df['Par_no_default']*model_df['Adj. All in Rate']).sum()/model_df['Par_no_default'].sum()
    Port_stats_df.loc['Max Moodys Rating Factor Test (NEW WARF)','Portfolio Stats'] = \
        (model_df['Par_no_default']*model_df['Adj. WARF NEW']).sum()/model_df['Par_no_default'].sum()
    Port_stats_df.loc['Max Moodys Rating Factor Test (Orig WARF)','Portfolio Stats'] = \
        (model_df['Par_no_default']*model_df['WARF']).sum()/model_df['Par_no_default'].sum()
    Port_stats_df.loc['Min Moodys Recovery Rate Test','Portfolio Stats'] = \
        (model_df['Par_no_default']*model_df['Moodys Recovery Rate']).sum()/model_df['Par_no_default'].sum()
        
    
    Port_stats_df.loc['Min S&P Recovery Rate Class A-1a','Portfolio Stats'] = \
        (model_df['Par_no_default']*model_df['S&P Recovery Rate (AAA)']).sum()/model_df['Par_no_default'].sum()
    Port_stats_df.loc['Moodys Diversity Test','Portfolio Stats'] = diversity_score(model_df, ind_avg_eu)
    #Port_stats_df.loc['WAP','Portfolio Stats'] = \
    #    model_df['Par_no_default']*model_df['Spread']/sum(model_df['Par_no_default'])
    Port_stats_df.loc['Total Portfolio Par (excl. Defaults)','Portfolio Stats'] = model_df['Par_no_default'].sum()
    Port_stats_df.loc['Total Portfolio Par','Portfolio Stats'] = model_df['Quantity'].sum()
    
    # current portfolio is Quantity + Add'l Amount (manual) TBA later
    #Port_stats_df['Current Portfolio'] = model_df['Par_no_default']*model_df['Spread']/sum(model_df['Par_no_default'])
    
    return Port_stats_df


Minimum Floating Spread Test - Without Libor Floors	3.28%
Minimum Floating Spread Test - WithLibor Floors (adj. All in Rate)	3.47%
Maximum Moody's Rating Factor Test	 2,908 
Maximum Moody's Rating Factor Test (NEW WARF)	 2,785 
Maximum Moody's Rating Factor Test (Orig WARF)	 2,908 
Minimum Weighted Average Moody's Recovery Rate Test	48.0%
Minimum Weighted Average S&P Recovery Rate Class A-1a	41.2%
Moody's Diversity Test	 63 
WAP (Current Positions use Actual purchase price, all others use Ask price)	99.74%


In [112]:
Port_stats(model_port)

,Portfolio Stats
Min Floating Spread Test - no Libor Floors,3.277878e-02
Min Floating Spread Test - With Libor Floors,3.465006e-02
Max Moodys Rating Factor Test (NEW WARF),2.784696e+03
Max Moodys Rating Factor Test (Orig WARF),2.908407e+03
Min Moodys Recovery Rate Test,4.796017e-01
Min S&P Recovery Rate Class A-1a,4.121602e-01
Moodys Diversity Test,6.272670e+01
WAP,NaN
Total Portfolio Par (excl. Defaults),2.263333e+08
Total Portfolio Par,2.263333e+08
